In [ ]:
# simulate some data

In [1]:
source("project_utils.R")
set.seed(607)

Loading required package: MASS

Loading required package: Matrix

Loading required package: mnormt



In [2]:
N <- 1000 # number of samples
p <- c(0.9, 0.1) # fraction in each component (null, non-null)
S <- matrix(c(0.0012,0.0005, 0.0005, 0.0010),2,2)
fixS <- nearPD(S)$mat # nearest positive definite matrix
cov.data.M1.sim <- model1Sim(N, p, fixS)


2 x 2 Matrix of class "dpoMatrix"
       [,1]  [,2]
[1,] 0.0012 5e-04
[2,] 0.0005 1e-03


In [ ]:
# try to see if we can set DL differently

In [3]:
start.time <- Sys.time()
# run stan to estimate the covariance
library(rstan)
options(warn=-1)
fit1_sim <- stan(
  file = "../../../tmp_models6/m1_1.stan",  # Stan program
  data = cov.data.M1.sim,    # named list of data
  chains = 1,             # number of Markov chains
  warmup = 200,          # number of warmup iterations per chain
  iter = 800,            # total number of iterations per chain
  cores = 200,              
  refresh = 200          # show progress every 'refresh' iterations
  )
end.time <- Sys.time()
time.taken <- end.time - start.time
time.taken

Loading required package: StanHeaders

Loading required package: ggplot2

rstan (Version 2.19.3, GitRev: 2e1f913d3ca3)

For execution on a local, multicore CPU with excess RAM we recommend calling
options(mc.cores = parallel::detectCores()).
To avoid recompilation of unchanged Stan programs, we recommend calling
rstan_options(auto_write = TRUE)

hash mismatch so recompiling; make sure Stan code ends with a blank line




SAMPLING FOR MODEL 'm1_1' NOW (CHAIN 1).
Chain 1: 
Chain 1: Gradient evaluation took 0 seconds
Chain 1: 1000 transitions using 10 leapfrog steps per transition would take 0 seconds.
Chain 1: Adjust your expectations accordingly!
Chain 1: 
Chain 1: 
Chain 1: Iteration:   1 / 800 [  0%]  (Warmup)
Chain 1: Iteration: 200 / 800 [ 25%]  (Warmup)
Chain 1: Iteration: 201 / 800 [ 25%]  (Sampling)
Chain 1: Iteration: 400 / 800 [ 50%]  (Sampling)
Chain 1: Iteration: 600 / 800 [ 75%]  (Sampling)
Chain 1: Iteration: 800 / 800 [100%]  (Sampling)
Chain 1: 
Chain 1:  Elapsed Time: 6.06 seconds (Warm-up)
Chain 1:                15.52 seconds (Sampling)
Chain 1:                21.58 seconds (Total)
Chain 1: 


Time difference of 5.659313 mins

In [9]:
fit1_sim@stanmodel

S4 class stanmodel 'm1_1' coded as follows:
data {
    int<lower=0> N; 
    int<lower=1> M; 
    matrix[N, M] B; 
    matrix[N, M] SE; 
    int<lower=1> K; 
}
transformed data{
    vector[M] zeros;
    vector[2] dl;
    matrix[M,M] SE_mat[N];
    zeros = rep_vector(0, M);

    dl[1] = 2;
    dl[2] = 1;

    for (n in 1:N) {
        SE_mat[n] = diag_matrix(to_vector(SE[n]));
    }
}

parameters {
    simplex[K] pi; 
    cholesky_factor_corr[M] L_Omega;
    vector<lower=0>[M] tau;
}

transformed parameters{
    matrix[M, M] Sigma;
    matrix[M, M] Sigmas[K];
    Sigma = diag_pre_multiply(tau, L_Omega)*diag_pre_multiply(tau, L_Omega)';

    Sigmas[1] = diag_matrix(rep_vector(0,M));
    Sigmas[2] = Sigma;
}

model {
    vector[K] ps; 
    pi ~ dirichlet(dl);
    tau ~ cauchy(0, 2.5);
    L_Omega ~ lkj_corr_cholesky(2);


    for (n in 1:N){
        target += log_mix(pi[1],
                            multi_normal_lpdf(B[n] | zeros, SE_mat[n] + Sigmas[1]),
                            multi_

In [6]:
print(fit1_sim)

Inference for Stan model: m1_1.
1 chains, each with iter=800; warmup=200; thin=1; 
post-warmup draws per chain=600, total post-warmup draws=600.

                 mean se_mean   sd    2.5%     25%     50%     75%   97.5%
pi[1]            0.90    0.00 0.02    0.86    0.89    0.90    0.91    0.93
pi[2]            0.10    0.00 0.02    0.07    0.09    0.10    0.11    0.14
L_Omega[1,1]     1.00     NaN 0.00    1.00    1.00    1.00    1.00    1.00
L_Omega[1,2]     0.00     NaN 0.00    0.00    0.00    0.00    0.00    0.00
L_Omega[2,1]     0.43    0.01 0.12    0.18    0.35    0.43    0.51    0.65
L_Omega[2,2]     0.89    0.00 0.06    0.76    0.86    0.90    0.94    0.98
tau[1]           0.03    0.00 0.00    0.03    0.03    0.03    0.04    0.04
tau[2]           0.04    0.00 0.00    0.03    0.03    0.04    0.04    0.05
Sigma[1,1]       0.00    0.00 0.00    0.00    0.00    0.00    0.00    0.00
Sigma[1,2]       0.00    0.00 0.00    0.00    0.00    0.00    0.00    0.00
Sigma[2,1]       0.00    0.00

In [ ]:
start.time <- Sys.time()
# run stan to estimate the covariance
options(warn=-1)
fit1_v2_sim <- stan(
  file = "../../../tmp_models3/m1_1.stan",  # Stan program
  data = cov.data.M1.sim,    # named list of data
  chains = 4,             # number of Markov chains
  warmup = 200,          # number of warmup iterations per chain
  iter = 800,            # total number of iterations per chain
  cores = 200,              
  refresh = 200          # show progress every 'refresh' iterations
  )
end.time <- Sys.time()
time.taken <- end.time - start.time
time.taken

In [ ]:
print(fit1_v2_sim, pars=c("pi", "Sigma"))

In [ ]:
print(fit1_sim, pars=c("pi", "Sigma"))

In [11]:
start.time <- Sys.time()
# run stan to estimate the covariance
options(warn=-1)
fit1_2_v2_sim <- stan(
  file = "../../../tmp_models3/m1_4.stan",  # Stan program
  data = cov.data.M1.sim,    # named list of data
  chains = 4,             # number of Markov chains
  warmup = 200,          # number of warmup iterations per chain
  iter = 800,            # total number of iterations per chain
  cores = 200,              
  refresh = 200          # show progress every 'refresh' iterations
  )
end.time <- Sys.time()
time.taken <- end.time - start.time
time.taken

Time difference of 3.063639 mins

In [12]:
print(fit1_2_v2_sim, pars=c("pi", "Sigma"))

Inference for Stan model: m1_4.
4 chains, each with iter=800; warmup=200; thin=1; 
post-warmup draws per chain=600, total post-warmup draws=2400.

           mean se_mean   sd 2.5%  25%  50%  75% 97.5% n_eff Rhat
pi[1]      0.52       0 0.03 0.47 0.50 0.52 0.54  0.57  1903    1
pi[2]      0.48       0 0.03 0.43 0.46 0.48 0.50  0.53  1903    1
Sigma[1,1] 0.00       0 0.00 0.00 0.00 0.00 0.00  0.00  2259    1
Sigma[1,2] 0.00       0 0.00 0.00 0.00 0.00 0.00  0.00  1943    1
Sigma[2,1] 0.00       0 0.00 0.00 0.00 0.00 0.00  0.00  1943    1
Sigma[2,2] 0.00       0 0.00 0.00 0.00 0.00 0.00  0.00  1852    1

Samples were drawn using NUTS(diag_e) at Fri May  1 17:18:42 2020.
For each parameter, n_eff is a crude measure of effective sample size,
and Rhat is the potential scale reduction factor on split chains (at 
convergence, Rhat=1).


In [ ]:
load("data/")